In [1]:
#top 55127 word, max count
textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/*");
counts = textFile.map(lambda line: (line.split("\t")[0],int(line.split("\t")[2])))\
                    .reduceByKey(lambda a, b: max(a,b))\
                    .sortBy(lambda a: a[1],ascending=False)\
                    .take(55127);
counts=sc.parallelize(counts).saveAsTextFile("hdfs://student13-x1:9000/ngram_results_1/top_55127");
print "finished"
"""for item in counts:
    print item;"""

finished


'for item in counts:\n    print item;'

In [3]:
#total words number
textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/*");
counts = textFile.map(lambda line: (line.split("\t")[0],int(line.split("\t")[2])))\
                    .reduceByKey(lambda a, b: max(a,b))\
                    .sortBy(lambda a: a[1],ascending=False);
print len(counts.collect());

5512730


In [6]:
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results/top_55127/*");
print len(textFile.collect())
newlist=[]
count=0
for item in textFile.collect():
    count+=1
    newlist.append(item)
    if count>=11025:
        break
newlist2=sc.parallelize(newlist).map(lambda line:line.replace('(','').replace(')','').split(',')[0]).collect();
print len(newlist2)
def isPopular_word(word):
    for w in newlist2:
        if word in w:
            return True
    return False
def isPopular_line(line):
    word=line.split("\t")[0]
    for w in newlist2:
        if word in w:
            return True
    return False



55127
11025
(u'Zinik', (False, {u'1985': u'16'}))


In [10]:
print "start";
textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/googlebooks-eng-fiction-all-1gram-20120701-z");

def reducefunction(a,b):
    a.update(b)
    return a
def mapfunction(line):
    word=line.split("\t")[0]
    year=int(line.split("\t")[1])
    wcount=int(line.split("\t")[2])
    a=word
    b={year:wcount}
    return (a,b)
p_counts=textFile.map(mapfunction);
p_counts=p_counts.reduceByKey(reducefunction)\
                    .take(5);
#p_counts = textFile.filter(isPopular).take(5);
#print len(p_counts)

print p_counts[0]

start
(u'Zinik', {1985: 16, 1987: 5, 1988: 4, 1991: 8, 1992: 10, 1993: 16, 1995: 44, 1996: 76, 1997: 10, 1998: 1, 1999: 2, 2000: 2, 2001: 10, 2002: 10, 2003: 16, 2004: 16, 2005: 14, 2006: 3, 2007: 1, 2008: 4, 2009: 7})


In [1]:
#step1
#11025 detail info (word,(max,dic))
textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/*");
def mapfunction(line):
    word=line.split("\t")[0]
    year=int(line.split("\t")[1])
    wcount=int(line.split("\t")[2])
    a=word
    tempMax=wcount
    b=(tempMax,{year:wcount})
    return (a,b)
def reducefuntion(a,b):
    tempMax=max(a[0],b[0])
    a[1].update(b[1])
    return (tempMax,a[1])
counts = textFile.map(mapfunction);
counts = counts.reduceByKey(reducefuntion);
counts = counts.sortBy(lambda a: a[1][0],ascending=False).take(11025);

counts=sc.parallelize(counts).saveAsTextFile("hdfs://student13-x1:9000/cache1_ngram_results_step1/top_11025_detailed");

In [3]:
counts=sc.parallelize(counts).saveAsTextFile("hdfs://student13-x1:9000/ngram_results_2/top_11025_detailed");

TypeError: 'PipelinedRDD' object is not iterable

In [2]:
#step2
#get study formatted source
unit=10000
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results_1/top_11025_detailed/*");
#print textFile.collect()[0];
def find_nearest(array,value):
    minabs=9999999
    result=0
    for v in array:
        tempabs=abs(v-value)
        if tempabs<minabs:
            minabs=tempabs
            result=v
    return result
def get2counts(dic):
    year4=-1
    for key in dic.keys():
        if dic[key]>=89794:
            year4=key
            break
    year3=year4-5
    year2=year3-10
    year1=year2-10
    year3=find_nearest(dic.keys(),year3)
    year2=find_nearest(dic.keys(),year2)
    year1=find_nearest(dic.keys(),year1)
    value3=dic[year3]/unit
    value2=dic[year2]/unit
    value1=dic[year1]/unit
    return (value1,value2,value3)
def mapfunction(line):
    line=line.replace('(','').replace(')','')
    word=line.split(',')[0].replace('u\'','').replace('\'','')
    maxcount=int(line.split(',')[1])
    dic_string='{'+line.split('{')[1]
    import ast
    dic= ast.literal_eval(dic_string)
    value_2=get2counts(dic)
    return (word,value_2)
    #return (word,(maxcount,dic))
counts=textFile.map(mapfunction);
counts=counts.reduceByKey(lambda a,b:a);
#counts=counts.take(5);
counts=counts.saveAsTextFile("hdfs://student13-x1:9000/ngram_results_1/11025_formatted_unit10000");
print "finished"
#print counts[0]
#print counts[1]
#for item in counts:
#    if 'kill' in item[0]:
#        print item

Py4JJavaError: An error occurred while calling o174.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://student13-x1:9000/ngram_results_1/11025_formatted_unit10000 already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1191)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$ano

In [39]:
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results/11025_formatted_unit10000");
print textFile.collect()[0]

(u'kill_VERB', (1, 2, 6))


In [24]:
#get total number and compute frequency
textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/*");
counts = textFile.map(lambda line: (int(line.split("\t")[1]),int(line.split("\t")[2])))\
                    .reduceByKey(lambda a, b: a+b)\
                    .sortBy(lambda a: a[1],ascending=False).collect();
#counts=sc.parallelize(counts).saveAsTextFile("hdfs://student13-x1:9000/ngram_results/year_counts");


Py4JJavaError: An error occurred while calling o835.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://student13-x1:9000/ngram_results/year_counts already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1191)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoo

In [25]:
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results/year_counts");
counts=textFile.collect();
year_f={}
for item in counts:
    year_f[item[0]]=item[1]
    print item

(2008, 13790808567)
(2007, 10401776121)
(2006, 8822750185)
(2004, 8561042762)
(2005, 8238348598)
(2003, 7430168916)
(2002, 6669648588)
(2001, 5910085555)
(2000, 5706074792)
(2009, 5541934147)
(1999, 4914319741)
(1998, 4545440716)
(1997, 4293207657)
(1996, 4228032019)
(1995, 4005306672)
(1994, 3915798737)
(1993, 3640121489)
(1992, 3637111283)
(1990, 3392380200)
(1991, 3367310379)
(1989, 3122962323)
(1988, 2946975237)
(1987, 2841397746)
(1986, 2692115243)
(1985, 2525988715)
(1984, 2421380424)
(1983, 2261789542)
(1982, 2191301282)
(1981, 2069121437)
(1980, 1992066746)
(1979, 1908817049)
(1978, 1858504180)
(1977, 1809766266)
(1976, 1785546223)
(1970, 1715307924)
(1972, 1681296079)
(1975, 1655844684)
(1971, 1651130374)
(1969, 1638727902)
(1974, 1633939711)
(1968, 1632327388)
(1973, 1629469672)
(1967, 1492931678)
(1966, 1392438269)
(1965, 1357600303)
(1963, 1218518917)
(1964, 1208129610)
(1962, 1171119682)
(1961, 1104374389)
(1960, 985190888)
(1959, 880749557)
(1958, 857664234)
(1957, 848561

In [6]:
textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/*");
textFile2 = sc.textFile("hdfs://student13-x1:9000/ngram_results/year_counts");
year_c_f=textFile2.collect();
year_f={}
for item in year_c_f:
    item=item.replace('(','').replace(')','');
    year=int(item.split(',')[0])
    y_count=int(item.split(',')[1])
    year_f[year]=y_count
def mapfunction(line):
    word=line.split("\t")[0]
    year=int(line.split("\t")[1])
    wcount=float(line.split("\t")[2])/year_f[year]
    a=word
    tempMax=wcount
    b=(tempMax,{year:wcount})
    return (a,b)
def reducefuntion(a,b):
    tempMax=max(a[0],b[0])
    a[1].update(b[1])
    return (tempMax,a[1])
counts2 = textFile.map(mapfunction);
counts2 = counts2.reduceByKey(reducefuntion);
counts2 = counts2.sortBy(lambda a: a[1][0],ascending=False).take(11025);

#print counts2[0]
#counts2 =sc.parallelize(counts).saveAsTextFile("hdfs://student13-x1:9000/ngram_results/top_11025_detailed_f");

(u'Zeno', (0.00010983217643440822, {1598: 0.00010983217643440822, 1708: 3.668591209615231e-07, 1717: 2.1743195466978608e-06, 1720: 6.747349978296025e-07, 1726: 1.9408152277915426e-07, 1728: 2.2983808596358123e-07, 1734: 2.733563900336447e-07, 1737: 1.7192836226543815e-07, 1739: 5.865734510208235e-07, 1745: 5.59213120034537e-07, 1748: 5.3558851805335e-07, 1750: 1.2350512484515544e-07, 1753: 2.68614299413615e-07, 1754: 3.8471323988050293e-07, 1760: 1.9036972847184888e-07, 1768: 1.0186080334152328e-07, 1770: 4.7500499942761895e-07, 1788: 1.3061259461984517e-07, 1790: 2.559502187062625e-07, 1794: 4.3113141764039945e-07, 1795: 5.6125894422253516e-08, 1796: 2.9812393165159634e-07, 1797: 1.0695680768583064e-07, 1801: 2.622261087627889e-08, 1804: 4.789243818151838e-08, 1806: 3.2611635130264324e-07, 1807: 6.195617451517847e-08, 1808: 4.021978503731491e-08, 1809: 9.649095772621143e-08, 1810: 1.3716022005917126e-07, 1811: 1.1737648875569917e-07, 1812: 5.211942671497182e-08, 1813: 2.42078687230369

In [4]:
#step3
#3 conditions ar
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results_1/11025_formatted_unit10000");
print textFile.collect()[0]
dictionary={}
allcounts=textFile.collect();
dic_ar={}
for item in allcounts:
    item=item.replace('(','').replace(')','')
    word=item.split(',')[0].replace('u\'','').replace('\'','')
    counts1=int(item.split(',')[1])
    counts2=int(item.split(',')[2])
    counts3=int(item.split(',')[3])
    counts=(counts1,counts2,counts3)
    dictionary[word]=counts
    if counts in dic_ar.keys():
        dic_ar[counts]+=1
    else:
        dic_ar[counts]=1
print dictionary['kill_VERB']
print dic_ar[(1,2,6)]
dic_ar_list=[]
for key in dic_ar.keys():
    dic_ar_list.append((key,dic_ar[key]))
dic_ar_rdd=sc.parallelize(dic_ar_list).sortBy(lambda a: a[1],ascending=False);
dic_ar_rdd.saveAsTextFile("hdfs://student13-x1:9000/ngram_results_2/ar_rules_11025_u10000");
print dic_ar_rdd.collect()[1]

(u'kill_VERB', (1, 2, 6))
(1, 2, 6)
372
((1, 2, 5), 912)


In [ ]:
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results/ar_rules_11025_u10000");
print textFile.collect()[0]

In [5]:
#step 4
#match ar
#detail info (word,(max,dic))
file_names=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
for file_name_letter in  file_names:
    textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/googlebooks-eng-fiction-all-1gram-20120701-"+file_name_letter);
    def mapfunction(line):
        word=line.split("\t")[0]
        year=int(line.split("\t")[1])
        wcount=int(line.split("\t")[2])
        a=word
        b=(wcount,{year:wcount})
        return (a,b)
    def reducefuntion(a,b):
        c={}
        a[1].update(b[1])
        len_a=len(a[1].keys())
        key_list=sorted(a[1].keys())
        c[key_list[len_a-1]]=a[1][key_list[len_a-1]]
        c[key_list[len_a-2]]=a[1][key_list[len_a-2]]    
        c[key_list[len_a-3]]=a[1][key_list[len_a-3]]    
        return (max(a[0],b[0]),c)
    def mapfunction2(line):
        list_y=sorted(line[1][1].keys())
        if len(list_y)<=2 or line[1][0]>=89794:
            return (line[0],[0,0,0])
        listv=[]

        listv.append(line[1][1][list_y[0]])
        listv.append(line[1][1][list_y[1]])
        listv.append(line[1][1][list_y[2]])

        return (line[0],listv)
    counts = textFile.map(mapfunction);
    counts = counts.reduceByKey(reducefuntion);
    print counts.collect()[0]

    counts =counts.map(mapfunction2)
    counts = counts.reduceByKey(lambda a,b:a)
    counts=counts.collect()

    print counts[0]

    limit=700
    arFile = sc.textFile("hdfs://student13-x1:9000/ngram_results_1/ar_rules_11025_u10000");
    arSet=arFile.collect()
    print arSet[0]
    print len(arSet)
    arSet_limit=[]
    for item in arSet:
        import ast
        ar=ast.literal_eval(item)
        if ar[1]>=limit:
            arSet_limit.append(ar)
    print len(arSet_limit)
    wlist=[]
    for item in counts:
        for ar in arSet_limit:
            if ar[0]==tuple(item[1]):
                neww=(item[0],item[1],ar[1])
                wlist.append(neww)
    print len(wlist)
    print wlist[0]
    sc.parallelize(wlist).saveAsTextFile("hdfs://student13-x1:9000/ngram_results_2/final_results/results_limit700_"+file_name_letter);
print "finished"
#for item in wlist:
#    print item

#counts = counts.sortBy(lambda a: a[1][0],ascending=True).take(5512730-11025);

#counts=sc.parallelize(counts).saveAsTextFile("hdfs://student13-x1:9000/ngram_results/notin_11025_detailed");

(u'Alhamhra_NOUN', (15, {2008: 15, 2009: 6, 2007: 2}))
(u'Antam', [4, 5, 6])
((1, 2, 4), 954)
221
2
457
(u'A32_NUM', [1, 2, 4], 954)
(u'Bardachd_NOUN', (16, {2008: 5, 2009: 10, 2007: 6}))
(u'Bardachd_NOUN', [6, 5, 10])
((1, 2, 4), 954)
221
2
386
(u'Brdhman', [1, 2, 4], 954)
(u'Chauntecleer_NOUN', (1379, {2008: 111, 2009: 62, 2007: 21}))
(u'Chauntecleer_NOUN', [21, 111, 62])
((1, 2, 4), 954)
221
2
595
(u'corporelles_X', [1, 2, 4], 954)
(u'd\xe9sirais_X', (6, {2008: 5, 2009: 6, 2007: 4}))
(u'd\xe9sirais_X', [4, 5, 6])
((1, 2, 4), 954)
221
2
361
(u'decapitate_ADJ', [1, 2, 4], 954)
(u'eigens_X', (13, {2008: 5, 2009: 3, 2007: 2}))
(u'eigens_X', [2, 5, 3])
((1, 2, 4), 954)
221
2
316
(u'egyenl\u0151_NOUN', [1, 2, 5], 912)
(u'fawn', (12211, {2008: 12211, 2009: 7427, 2007: 7858}))
(u'fawn', [7858, 12211, 7427])
((1, 2, 4), 954)
221
2
366
(u'Frilly_ADJ', [1, 2, 4], 954)
(u'Glendhu', (47, {2008: 47, 2009: 2, 2007: 36}))
(u'Glendhu', [36, 47, 2])
((1, 2, 4), 954)
221
2
245
(u'geschildert_X', [1, 2

In [35]:
textFile = sc.textFile("hdfs://student13-x1:9000/ngram_results/final_results/*/*");
counts=textFile.collect();
print len(counts)
for i in range(0,10):
    print counts[i]



7549
(u'Ambulant_ADJ', [1, 2, 4], 954)
(u'ahsorhing', [1, 2, 4], 954)
(u'anqther', [1, 2, 4], 954)
(u'awys', [1, 2, 4], 954)
(u'allit_VERB', [1, 2, 4], 954)
(u'anflygning_VERB', [1, 2, 4], 954)
(u'A32_NUM', [1, 2, 4], 954)
(u'arrebata', [1, 2, 4], 954)
(u'allaway', [1, 2, 5], 912)
(u'Arraign_VERB', [1, 2, 4], 954)


In [50]:
count_124=0
count_125=0
file_names=["o","p","w","g"]
for file_name_letter in  file_names:
    textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/googlebooks-eng-all-1gram-20120701-"+file_name_letter);
    def mapfunction(line):
        word=line.split("\t")[0]
        year=int(line.split("\t")[1])
        wcount=int(line.split("\t")[2])
        a=word
        b=(wcount,{year:wcount})
        return (a,b)
    def reducefuntion(a,b):
        a[1].update(b[1])
        return (max(a[0],b[0]),a[1])


    counts = textFile.map(mapfunction);
    counts = counts.reduceByKey(reducefuntion);

    unit=10000
    def find_nearest(array,value):
        minabs=9999999
        result=0
        for v in array:
            tempabs=abs(v-value)
            if tempabs<minabs:
                minabs=tempabs
                result=v
        return result
    def get2counts(dic,maxv):
        year4=-1
        for key in dic.keys():
            if maxv<89794:
                if dic[key]>=maxv:
                    year4=key
                    break
            elif dic[key]>=89794:
                year4=key
                break
                
        year3=year4-5
        year2=year3-10
        year1=year2-10
        year3=find_nearest(dic.keys(),year3)
        year2=find_nearest(dic.keys(),year2)
        year1=find_nearest(dic.keys(),year1)
        value3=dic[year3]/unit
        value2=dic[year2]/unit
        value1=dic[year1]/unit
        return (value1,value2,value3)
    def mapfunction2(line):
        word=line[0]
        maxcount=line[1][0]

        dic= line[1][1]
        value_2=get2counts(dic,maxcount)
        return (line[0],value_2)

    counts = counts.map(mapfunction2);
    counts=  counts.reduceByKey(lambda a,b:a);
    def isequal(a,b):
        for i in range(0,3):
            if a[i]!=b[i]:
                return False
        return True
    def mapfunction3(line):
        if isequal(line[1],(1,2,4)):
            return ('124',1)
        if isequal(line[1],(1,2,5)):
            return ('125',1)
        return ('0',0)
    def reducefunction3(a,b):
        return a+b
    counts = counts.map(mapfunction3);
    counts=  counts.reduceByKey(reducefunction3);
    print file_name_letter
    print counts.collect()[0]
    if len(counts.collect())>1:
        print counts.collect()[1]
    if len(counts.collect())>2:
        print counts.collect()[2]
print "finished"

o
('124', 59)
('125', 24)
('0', 0)
p
('0', 0)
('124', 230)
('125', 107)
w
('0', 0)
('124', 74)
('125', 51)
g
('0', 0)
('124', 102)
('125', 31)
finished


In [2]:
#for presetation
#z in the step4
print "start"
file_names=["z"]
for file_name_letter in  file_names:
    textFile = sc.textFile("hdfs://student13-x1:9000/ngram/complete/googlebooks-eng-fiction-all-1gram-20120701-"+file_name_letter);
    def mapfunction(line):
        word=line.split("\t")[0]
        year=int(line.split("\t")[1])
        wcount=int(line.split("\t")[2])
        a=word
        b=(wcount,{year:wcount})
        return (a,b)
    def reducefuntion(a,b):
        c={}
        a[1].update(b[1])
        len_a=len(a[1].keys())
        key_list=sorted(a[1].keys())
        c[key_list[len_a-1]]=a[1][key_list[len_a-1]]
        c[key_list[len_a-2]]=a[1][key_list[len_a-2]]    
        c[key_list[len_a-3]]=a[1][key_list[len_a-3]]    
        return (max(a[0],b[0]),c)
    def mapfunction2(line):
        list_y=sorted(line[1][1].keys())
        if len(list_y)<=2 or line[1][0]>=89794:
            return (line[0],[0,0,0])
        listv=[]

        listv.append(line[1][1][list_y[0]])
        listv.append(line[1][1][list_y[1]])
        listv.append(line[1][1][list_y[2]])

        return (line[0],listv)
    counts = textFile.map(mapfunction);
    counts = counts.reduceByKey(reducefuntion);
    #print counts.collect()[0]

    counts =counts.map(mapfunction2)
    counts = counts.reduceByKey(lambda a,b:a)
    counts=counts.collect()

    #print counts[0]

    limit=700
    arFile = sc.textFile("hdfs://student13-x1:9000/ngram_results_1/ar_rules_11025_u10000");
    arSet=arFile.collect()
    #print arSet[0]
    #print len(arSet)
    arSet_limit=[]
    for item in arSet:
        import ast
        ar=ast.literal_eval(item)
        if ar[1]>=limit:
            arSet_limit.append(ar)
    #print len(arSet_limit)
    wlist=[]
    for item in counts:
        for ar in arSet_limit:
            if ar[0]==tuple(item[1]):
                neww=(item[0],item[1],ar[1])
                wlist.append(neww)
    print len(wlist)
    #print wlist[0]
    sc.parallelize(wlist).saveAsTextFile("hdfs://student13-x1:9000/ngram_results_5/final_results/results_limit700_"+file_name_letter);
print "finished"


start
37
finished
